In [2]:
import sys, os
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(r"C:\Users\super\DAG")
sys.path.append(r"c:\Users\super\DAG\golemMain\src")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from MEC import is_in_markov_equiv_class
print(os.getcwd())
from SCM_data import generate_scm_from_BN 
from numpy.linalg import inv
from scipy.linalg import sqrtm
from nodag.nodag_gumbel_softmax import train_gumbel_sgd_dag
from nodag.nodag_gs_findbest import nodag_fb_l_dag
from golemMain.src.golem import golem
from coordinate_descent.coordinate import dag_coordinate_descent, weight_to_adjacency
from coordinate_descent.coordinate0 import dag_coordinate_descent_l0

c:\Users\super\DAG

c:\Users\super\DAG
c:\Users\super\DAG


In [3]:
experiments = []

# ----------- Experiment 1 -----------
experiments.append({
    "name": "d=3, A→B←C",
    "B_true": np.array([
        [0, 1, 0],
        [0, 0, 0],
        [0, 2, 0]
    ]),
    "N": np.array([1, 2, 3]),
    "Omega_true": np.diag([1, 2, 3])
})

# ----------- Experiment 2 -----------
experiments.append({
    "name": "d=3, A→B→C",
    "B_true": np.array([
        [0, 1, 0],
        [0, 0, 3],
        [0, 0, 0]
    ]),
    "N": np.array([1, 3, 4]),
    "Omega_true": np.diag([1, 3, 4])
})

# ----------- Experiment 3 -----------
experiments.append({
    "name": "d=3, A→B→C + A→C",
    "B_true": np.array([
        [0, 1, 2],
        [0, 0, 3],
        [0, 0, 0]
    ]),
    "N": np.array([5, 4, 3]),
    "Omega_true": np.diag([5, 4, 3])
})

# ----------- Experiment 4 -----------
experiments.append({
    "name": "d=4, A→B, B→C, B→D",
    "B_true": np.array([
        [0, 2, 0, 0],
        [0, 0, 3, 4],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    "N": np.array([1, 4, 3, 2]),
    "Omega_true": np.diag([1, 4, 3, 2])
})

# ----------- Experiment 5 -----------
experiments.append({
    "name": "d=4, A→C, A→D, B→C, B→D",
    "B_true": np.array([
        [0, 0, 2, 3],
        [0, 0, 3, 4],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ]),
    "N": np.array([2, 4, 3, 5]),
    "Omega_true": np.diag([2, 4, 3, 5])
})

# ----------- Experiment 6 -----------
experiments.append({
    "name": "d=4, A→D, B→D, C→D",
    "B_true": np.array([
        [0, 0, 0, 1],
        [0, 0, 0, 3],
        [0, 0, 0, 5],
        [0, 0, 0, 0]
    ]),
    "N": np.array([5, 4, 3, 2]),
    "Omega_true": np.diag([5, 4, 3, 2])
})

# ----------- Experiment 7 -----------
experiments.append({
    "name": "d=5, e=4, |v|=0",
    "B_true": np.array([
        [0, 1, 0, 2, 0],
        [0, 0, 3, 0, 4],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})

# ----------- Experiment 8 -----------
experiments.append({
    "name": "d=5, e=4, |v|=1",
    "B_true": np.array([
        [0, 0, 1, 2, 0],
        [0, 0, 0, 2, 3],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})

# ----------- Experiment 9 -----------
experiments.append({
    "name": "d=5, e=4, |v|=2",
    "B_true": np.array([
        [0, 0, 0, 1, 0],
        [0, 0, 0, 2, 3],
        [0, 0, 0, 0, 4],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ]),
    "N": np.array([1, 2, 3, 2, 1]),
    "Omega_true": np.diag([1, 2, 3, 2, 1])
})


In [4]:
def run_one_coordinate0(B_true, N, name=None,
                       n_samples=10000,
                       threshold=0.05,
                       T=300,
                       n_repeats=100):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running: {name}")

    G_true = weight_to_adjacency(B_true, threshold)
    N = np.array(N)

    data, G, B, Sigma = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=42
    )
    R_hat = np.cov(data.T)
    # print("R_hat =\n", np.array2string(
    #     R_hat, formatter={'float_kind': lambda x: f"{x:.2f}"}
    # ))

    d = R_hat.shape[0]

    Omega_true = np.diag(N)
    # print("inv(sqrtm(Omega_true)) =\n", inv(sqrtm(Omega_true)))

    Theta_hat = inv(R_hat)
    A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
    # print("A_true =\n", np.array2string(
    #     A_true, formatter={'float_kind': lambda x: f"{x:.2f}"}
    # ))

    likelihood_true = -2 * np.log(np.linalg.det(A_true)) \
                      + np.trace(A_true.T @ R_hat @ A_true)
    # print("likelihood_true =", likelihood_true)

    correct = 0
    for seed in range(n_repeats):
        A_opt, G_opt, fval = dag_coordinate_descent_l0(
            R_hat, T=T, seed=seed, threshold=threshold
        )
        if is_in_markov_equiv_class(G_true, G_opt):
            correct += 1

    print("Correct rate =", correct / n_repeats)

print("N_samples = 10000")
for exp in experiments:
    run_one_coordinate0(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=10000,
        threshold=0.05,
        T=1000,
        n_repeats=100
    )

print("N_samples = 1000")
for exp in experiments:
    run_one_coordinate0(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=1000,
        threshold=0.05,
        T=1000,
        n_repeats=100
    )

print("N_samples = 200")
for exp in experiments:
    run_one_coordinate0(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=200,
        threshold=0.05,
        T=1000,
        n_repeats=100
    )

N_samples = 10000
Running: d=3, A→B←C
Correct rate = 0.78
Running: d=3, A→B→C
Correct rate = 0.64
Running: d=3, A→B→C + A→C
Correct rate = 0.88
Running: d=4, A→B, B→C, B→D
Correct rate = 0.24
Running: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.45
Running: d=4, A→D, B→D, C→D
Correct rate = 0.84
Running: d=5, e=4, |v|=0
Correct rate = 0.28
Running: d=5, e=4, |v|=1
Correct rate = 0.56
Running: d=5, e=4, |v|=2
Correct rate = 0.6
N_samples = 1000
Running: d=3, A→B←C
Correct rate = 0.78
Running: d=3, A→B→C
Correct rate = 0.62
Running: d=3, A→B→C + A→C
Correct rate = 0.84
Running: d=4, A→B, B→C, B→D
Correct rate = 0.15
Running: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.44
Running: d=4, A→D, B→D, C→D
Correct rate = 0.84
Running: d=5, e=4, |v|=0
Correct rate = 0.29
Running: d=5, e=4, |v|=1
Correct rate = 0.58
Running: d=5, e=4, |v|=2
Correct rate = 0.57
N_samples = 200
Running: d=3, A→B←C
Correct rate = 0.78
Running: d=3, A→B→C
Correct rate = 0.65
Running: d=3, A→B→C + A→C
Correct rate = 0.82
Runn

In [ ]:
def run_one_coordinate0(B_true, N, name=None,
                       n_samples=10000,
                       threshold=0.05,
                       T=300,
                       n_repeats=100):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running: {name}")

    G_true = weight_to_adjacency(B_true, threshold)
    N = np.array(N)

    data, G, B, Sigma = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=42
    )
    R_hat = np.cov(data.T)
    # print("R_hat =\n", np.array2string(
    #     R_hat, formatter={'float_kind': lambda x: f"{x:.2f}"}
    # ))

    d = R_hat.shape[0]

    Omega_true = np.diag(N)
    # print("inv(sqrtm(Omega_true)) =\n", inv(sqrtm(Omega_true)))

    Theta_hat = inv(R_hat)
    A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
    # print("A_true =\n", np.array2string(
    #     A_true, formatter={'float_kind': lambda x: f"{x:.2f}"}
    # ))

    likelihood_true = -2 * np.log(np.linalg.det(A_true)) \
                      + np.trace(A_true.T @ R_hat @ A_true)
    # print("likelihood_true =", likelihood_true)

    correct = 0
    for seed in range(n_repeats):
        A_opt, G_opt, fval = dag_coordinate_descent_l0(
            R_hat, T=T, seed=seed, threshold=threshold
        )
        if is_in_markov_equiv_class(G_true, G_opt):
            correct += 1

    print("Correct rate =", correct / n_repeats)

print("N_samples = 10000")
for exp in experiments:
    run_one_coordinate0(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=10000,
        threshold=0.05,
        T=5000,
        n_repeats=100
    )

print("N_samples = 1000")
for exp in experiments:
    run_one_coordinate0(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=1000,
        threshold=0.05,
        T=5000,
        n_repeats=100
    )

print("N_samples = 200")
for exp in experiments:
    run_one_coordinate0(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=200,
        threshold=0.05,
        T=5000,
        n_repeats=100
    )

N_samples = 10000
Running: d=3, A→B←C
Correct rate = 0.78
Running: d=3, A→B→C
Correct rate = 0.64
Running: d=3, A→B→C + A→C
Correct rate = 0.88
Running: d=4, A→B, B→C, B→D
Correct rate = 0.38
Running: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.45
Running: d=4, A→D, B→D, C→D
Correct rate = 0.84
Running: d=5, e=4, |v|=0
Correct rate = 0.36
Running: d=5, e=4, |v|=1
Correct rate = 0.56
Running: d=5, e=4, |v|=2
Correct rate = 0.68
N_samples = 1000
Running: d=3, A→B←C
Correct rate = 0.78
Running: d=3, A→B→C
Correct rate = 0.62
Running: d=3, A→B→C + A→C
Correct rate = 0.84
Running: d=4, A→B, B→C, B→D
Correct rate = 0.32
Running: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.44
Running: d=4, A→D, B→D, C→D
Correct rate = 0.84
Running: d=5, e=4, |v|=0
Correct rate = 0.35
Running: d=5, e=4, |v|=1
Correct rate = 0.58
Running: d=5, e=4, |v|=2
Correct rate = 0.65
N_samples = 200
Running: d=3, A→B←C
Correct rate = 0.78
Running: d=3, A→B→C
Correct rate = 0.65
Running: d=3, A→B→C + A→C
Correct rate = 0.82
Run

: 

In [7]:
def run_one_coordinate(B_true, N, name=None,
                       n_samples=10000,
                       threshold=0.05,
                       T=300,
                       n_repeats=100):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running: {name}")

    G_true = weight_to_adjacency(B_true, threshold)
    N = np.array(N)

    data, G, B, Sigma = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=42
    )
    R_hat = np.cov(data.T)
    print("R_hat =\n", np.array2string(
        R_hat, formatter={'float_kind': lambda x: f"{x:.2f}"}
    ))

    d = R_hat.shape[0]

    Omega_true = np.diag(N)
    print("inv(sqrtm(Omega_true)) =\n", inv(sqrtm(Omega_true)))

    Theta_hat = inv(R_hat)
    A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
    print("A_true =\n", np.array2string(
        A_true, formatter={'float_kind': lambda x: f"{x:.2f}"}
    ))

    likelihood_true = -2 * np.log(np.linalg.det(A_true)) \
                      + np.trace(A_true.T @ R_hat @ A_true)
    print("likelihood_true =", likelihood_true)

    correct = 0
    for seed in range(n_repeats):
        A_opt, G_opt, fval = dag_coordinate_descent(
            R_hat, T=T, seed=seed, threshold=threshold
        )
        if is_in_markov_equiv_class(G_true, G_opt):
            correct += 1

    print("Correct rate =", correct / n_repeats)


for exp in experiments:
    run_one_coordinate(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=10000,
        threshold=0.05,
        T=3000,
        n_repeats=100
    )

Running: d=3, A→B←C
R_hat =
 [[1.02 1.06 0.02]
 [1.06 15.32 6.09]
 [0.02 6.09 3.02]]
inv(sqrtm(Omega_true)) =
 [[1.         0.         0.        ]
 [0.         0.70710677 0.        ]
 [0.         0.         0.57735026]]
A_true =
 [[1.00 -0.71 0.00]
 [0.00 0.71 0.00]
 [0.00 -1.41 0.58]]
likelihood_true = 4.816664771649337
Correct rate = 0.79
Running: d=3, A→B→C
R_hat =
 [[1.02 1.03 3.11]
 [1.03 4.04 12.20]
 [3.11 12.20 40.81]]
inv(sqrtm(Omega_true)) =
 [[1.         0.         0.        ]
 [0.         0.57735026 0.        ]
 [0.         0.         0.5       ]]
A_true =
 [[1.00 -0.58 0.00]
 [0.00 0.58 -1.50]
 [0.00 0.00 0.50]]
likelihood_true = 5.509811952486
Correct rate = 0.81
Running: d=3, A→B→C + A→C
R_hat =
 [[5.08 5.12 25.56]
 [5.12 9.15 37.77]
 [25.56 37.77 167.78]]
inv(sqrtm(Omega_true)) =
 [[0.4472136  0.         0.        ]
 [0.         0.5        0.        ]
 [0.         0.         0.57735026]]
A_true =
 [[0.45 -0.50 -1.15]
 [0.00 0.50 -1.73]
 [0.00 0.00 0.58]]
likelihood_true 

In [5]:
import ast
def run_one_gs(B_true, N, name=None,
                       n_samples=10000,
                       threshold=0.05,
                       T=300,
                       n_repeats=100):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running: {name}")

    G_true = weight_to_adjacency(B_true, threshold)
    N = np.array(N)

    data, G, B, Sigma = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=42
    )
    R_hat = np.cov(data.T)
    print("R_hat =\n", np.array2string(
        R_hat, formatter={'float_kind': lambda x: f"{x:.2f}"}
    ))

    d = R_hat.shape[0]

    Omega_true = np.diag(N)
    print("inv(sqrtm(Omega_true)) =\n", inv(sqrtm(Omega_true)))

    Theta_hat = inv(R_hat)
    A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
    print("A_true =\n", np.array2string(
        A_true, formatter={'float_kind': lambda x: f"{x:.2f}"}
    ))

    likelihood_true = -2 * np.log(np.linalg.det(A_true)) \
                      + np.trace(A_true.T @ R_hat @ A_true)
    print("likelihood_true =", likelihood_true)

    correct = 0
    for seed in range(n_repeats):
        np.random.seed(seed)
        B_init = np.random.randn(*R_hat.shape)
        B_final, G_final, info = train_gumbel_sgd_dag(
            Rhat_np=R_hat,
            B_init=B_init
        )
        if np.isnan(G_final).any(): continue
        #best_G = np.array(ast.literal_eval(best_G))
        #print("best_G =", best_G)
        if is_in_markov_equiv_class(G_true, G_final):
            correct += 1

    print("Correct rate =", correct / n_repeats)


for exp in experiments:
    run_one_gs(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=10000,
        threshold=0.05,
        T=3000,
        n_repeats=100
    )

Running: d=3, A→B←C
R_hat =
 [[1.02 1.06 0.02]
 [1.06 15.32 6.09]
 [0.02 6.09 3.02]]
inv(sqrtm(Omega_true)) =
 [[1.         0.         0.        ]
 [0.         0.70710677 0.        ]
 [0.         0.         0.57735026]]
A_true =
 [[1.00 -0.71 0.00]
 [0.00 0.71 0.00]
 [0.00 -1.41 0.58]]
likelihood_true = 4.816664771649337
Correct rate = 0.09
Running: d=3, A→B→C
R_hat =
 [[1.02 1.03 3.11]
 [1.03 4.04 12.20]
 [3.11 12.20 40.81]]
inv(sqrtm(Omega_true)) =
 [[1.         0.         0.        ]
 [0.         0.57735026 0.        ]
 [0.         0.         0.5       ]]
A_true =
 [[1.00 -0.58 0.00]
 [0.00 0.58 -1.50]
 [0.00 0.00 0.50]]
likelihood_true = 5.509811952486
Correct rate = 0.18
Running: d=3, A→B→C + A→C
R_hat =
 [[5.08 5.12 25.56]
 [5.12 9.15 37.77]
 [25.56 37.77 167.78]]
inv(sqrtm(Omega_true)) =
 [[0.4472136  0.         0.        ]
 [0.         0.5        0.        ]
 [0.         0.         0.57735026]]
A_true =
 [[0.45 -0.50 -1.15]
 [0.00 0.50 -1.73]
 [0.00 0.00 0.58]]
likelihood_true 

In [ ]:
import ast
def run_one_gs_fb(B_true, N, name=None,
                       n_samples=10000,
                       threshold=0.05,
                       T=300,
                       n_repeats=100):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running: {name}")

    G_true = weight_to_adjacency(B_true, threshold)
    N = np.array(N)

    data, G, B, Sigma = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=42
    )
    R_hat = np.cov(data.T)
    print("R_hat =\n", np.array2string(
        R_hat, formatter={'float_kind': lambda x: f"{x:.2f}"}
    ))

    d = R_hat.shape[0]

    Omega_true = np.diag(N)
    print("inv(sqrtm(Omega_true)) =\n", inv(sqrtm(Omega_true)))

    Theta_hat = inv(R_hat)
    A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
    print("A_true =\n", np.array2string(
        A_true, formatter={'float_kind': lambda x: f"{x:.2f}"}
    ))

    likelihood_true = -2 * np.log(np.linalg.det(A_true)) \
                      + np.trace(A_true.T @ R_hat @ A_true)
    print("likelihood_true =", likelihood_true)

    correct = 0
    for seed in range(n_repeats):
        best = nodag_fb_l_dag(R_hat = R_hat,lam = 0.5, tau_start=0.2, tau_end=0.2, times = 10)
        best_G = best["G"]
        #best_G = np.array(ast.literal_eval(best_G))
        #print("best_G =", best_G)
        if is_in_markov_equiv_class(G_true, best_G):
            correct += 1

    print("Correct rate =", correct / n_repeats)


for exp in experiments:
    run_one_gs_fb(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=10000,
        threshold=0.05,
        T=3000,
        n_repeats=1
    )

Running: d=3, A→B←C
R_hat =
 [[1.02 1.06 0.02]
 [1.06 15.32 6.09]
 [0.02 6.09 3.02]]
inv(sqrtm(Omega_true)) =
 [[1.         0.         0.        ]
 [0.         0.70710677 0.        ]
 [0.         0.         0.57735026]]
A_true =
 [[1.00 -0.71 0.00]
 [0.00 0.71 0.00]
 [0.00 -1.41 0.58]]
likelihood_true = 4.816664771649337
t =  0
Valid likelihoods: [32.30139186341072, 4.8161919538386835, 4.816195725399055, 4.8915450510001035, 4.816195941445485, 6.776601694746596, 4.816195257327397, 5.2341164100976245, 4.816190472725637, 4.8161871400894665]
Correct rate = 1.0
Running: d=3, A→B→C
R_hat =
 [[1.02 1.03 3.11]
 [1.03 4.04 12.20]
 [3.11 12.20 40.81]]
inv(sqrtm(Omega_true)) =
 [[1.         0.         0.        ]
 [0.         0.57735026 0.        ]
 [0.         0.         0.5       ]]
A_true =
 [[1.00 -0.58 0.00]
 [0.00 0.58 -1.50]
 [0.00 0.00 0.50]]
likelihood_true = 5.509811952486
t =  0
Valid likelihoods: [5.509336437195527, 5.50933844764676, 5.542134463802597, 30.74660804513807, 5.86263840049

ValueError: min() iterable argument is empty

In [5]:
def run_one_golem(B_true, N, name=None,
                       n_samples=10000,
                       threshold=0.05,
                       T=300,
                       n_repeats=100):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running: {name}")

    G_true = weight_to_adjacency(B_true, threshold)
    N = np.array(N)

    data, G, B, Sigma = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=42
    )
    R_hat = np.cov(data.T)
    correct = 0
    for seed in range(n_repeats):
        np.random.seed(seed)
        B_init = np.random.randn(*R_hat.shape)
        B_est = golem(data, lambda_1=2e-2, lambda_2=5.0,
                equal_variances=True, num_iter=1e+4, seed=seed, B_init=B_init)
        G_est = weight_to_adjacency(B_est, 0.1)
        if is_in_markov_equiv_class(G_true, G_est):
            correct += 1
    print("Correct rate =", correct / n_repeats)

for exp in experiments:
    run_one_golem(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=10000,
        threshold=0.05,
        T=300,
        n_repeats=100
    )

Running: d=3, A→B←C
Correct rate = 0.6
Running: d=3, A→B→C
Correct rate = 0.56
Running: d=3, A→B→C + A→C
Correct rate = 1.0
Running: d=4, A→B, B→C, B→D
Correct rate = 0.02
Running: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.01
Running: d=4, A→D, B→D, C→D
Correct rate = 0.35
Running: d=5, e=4, |v|=0
Correct rate = 0.0
Running: d=5, e=4, |v|=1
Correct rate = 0.06
Running: d=5, e=4, |v|=2
Correct rate = 0.08


In [8]:
def run_one_golem_nv(B_true, N, name=None,
                       n_samples=10000,
                       threshold=0.05,
                       T=300,
                       n_repeats=100):
    if name is None:
        name = "Unnamed experiment"
    print("=" * 80)
    print(f"Running: {name}")

    G_true = weight_to_adjacency(B_true, threshold)
    N = np.array(N)

    data, G, B, Sigma = generate_scm_from_BN(
        B_true.T, n_samples=n_samples, N=N, seed=42
    )
    R_hat = np.cov(data.T)
    correct = 0
    for seed in range(n_repeats):
        np.random.seed(seed)
        B_init = np.random.randn(*R_hat.shape)
        B_est = golem(data, lambda_1=2e-2, lambda_2=5.0,
                equal_variances=True, num_iter=5000, seed=seed, B_init=B_init)
        B_final = golem(data, lambda_1=2e-2, lambda_2=5.0,
                equal_variances=False, num_iter=5000, seed=seed, B_init=B_est)
        G_est = weight_to_adjacency(B_final, 0.1)
        if is_in_markov_equiv_class(G_true, G_est):
            correct += 1
    print("Correct rate =", correct / n_repeats)

for exp in experiments:
    run_one_golem_nv(
        B_true=exp["B_true"],
        N=exp["N"],
        name=exp["name"],
        n_samples=10000,
        threshold=0.05,
        T=300,
        n_repeats=30
    )

Running: d=3, A→B←C
Correct rate = 0.5
Running: d=3, A→B→C
Correct rate = 0.5666666666666667
Running: d=3, A→B→C + A→C
Correct rate = 1.0
Running: d=4, A→B, B→C, B→D
Correct rate = 0.2
Running: d=4, A→C, A→D, B→C, B→D
Correct rate = 0.03333333333333333
Running: d=4, A→D, B→D, C→D
Correct rate = 0.23333333333333334
Running: d=5, e=4, |v|=0
Correct rate = 0.0
Running: d=5, e=4, |v|=1
Correct rate = 0.0
Running: d=5, e=4, |v|=2
Correct rate = 0.0
